# Boilerplate ingestion specifications for the data generator

<!--
  ~ Licensed to the Apache Software Foundation (ASF) under one
  ~ or more contributor license agreements.  See the NOTICE file
  ~ distributed with this work for additional information
  ~ regarding copyright ownership.  The ASF licenses this file
  ~ to you under the Apache License, Version 2.0 (the
  ~ "License"); you may not use this file except in compliance
  ~ with the License.  You may obtain a copy of the License at
  ~
  ~   http://www.apache.org/licenses/LICENSE-2.0
  ~
  ~ Unless required by applicable law or agreed to in writing,
  ~ software distributed under the License is distributed on an
  ~ "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
  ~ KIND, either express or implied.  See the License for the
  ~ specific language governing permissions and limitations
  ~ under the License.
  -->

This notebook contains some pattern ingestions that can read from the data generator.

## SQL-based ingestion example

In this example, the EXTERN statement uses the `http` type ingestion to read directly from the `file` endpoint of the data generator API.

In [ ]:
sql = '''
REPLACE INTO "clicks" OVERWRITE ALL
WITH "ext" AS (SELECT *
FROM TABLE(
  EXTERN(
    '{"type":"http","uris":["http://datagen:9999/file/clicks.json"]}',
    '{"type":"json"}'
  )
) EXTEND ("time" VARCHAR, "user_id" VARCHAR, "event_type" VARCHAR, "client_ip" VARCHAR, "client_device" VARCHAR, "client_lang" VARCHAR, "client_country" VARCHAR, "referrer" VARCHAR, "keyword" VARCHAR, "product" VARCHAR))
SELECT
  TIME_PARSE("time") AS "__time",
  "user_id",
  "event_type",
  "client_ip",
  "client_device",
  "client_lang",
  "client_country",
  "referrer",
  "keyword",
  "product"
FROM "ext"
PARTITIONED BY DAY
'''  

druid.display.run_task(sql)
print("Waiting for segment avaialbility ...")
druid.sql.wait_until_ready('clicks')
print("Data is available for query.")

## Kafka-based native ingestion example

This native ingestion specification connects to the learn-druid Kafka container (`bootstrap.servers`) to read data from a topic that the data generator is writing to.

In [ ]:
ingestion_spec ={
  "type": "kafka",
  "spec": {
    "ioConfig": {
      "type": "kafka",
      "consumerProperties": {
        "bootstrap.servers": "kafka:9092"
      },
      "topic": "custom_data",
      "inputFormat": {
        "type": "json"
      },
      "useEarliestOffset": True
    },
    "tuningConfig": {
      "type": "kafka",
      "maxRowsInMemory": 100000,
      "resetOffsetAutomatically": False
    },
    "dataSchema": {
      "dataSource": "custom_data",
      "timestampSpec": {
        "column": "time",
        "format": "iso"
      },
      "dimensionsSpec": {
        "dimensions": [
          "random_string_column",
          {
            "type": "long",
            "name": "distributed_number"
          }
        ]
      },
      "granularitySpec": {
        "queryGranularity": "none",
        "rollup": False,
        "segmentGranularity": "hour"
      }
    }
  }
}

headers = {
  'Content-Type': 'application/json'
}

druid.rest.post("/druid/indexer/v1/supervisor", json.dumps(ingestion_spec), headers=headers)